In [225]:
import pandas as pd
import numpy as np

import math,datetime
import time

import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split


In [226]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt

In [227]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

df=pd.read_csv('historical_data.csv')


In [228]:
corr_matrix = df.corr()
with open('corr.npy', 'wb') as f:
    np.save(f, corr_matrix)

In [229]:
f

<_io.BufferedWriter name='corr.npy'>

In [230]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt

In [231]:
# data = df.filter(['elec_price'])

data = df.filter(['elec_price',
 
 ])

dataset = data.values
dataset

array([[42.34],
       [38.89],
       [38.82],
       ...,
       [97.02],
       [96.01],
       [84.36]])

In [232]:
training_data_len = math.ceil( len(dataset) * .8 )

In [233]:
scaler =StandardScaler()

scaled_data = scaler.fit_transform(dataset)


In [234]:


#Create the scaled training data set
train_data = scaled_data[0:training_data_len , :]
#Split the data into x_train and y_train data sets
x_trainl = []
y_trainl = []

#hourly prediction
training_window = 24
prediction_time=1
#daily prediction
# training_window = 24*7
# prediction_time=24*1
#weekly prediction
# training_window = 24*7*3
# prediction_time=24*7

for i in range(training_window, len(train_data)-prediction_time):
    x_trainl.append(train_data[i-training_window:i, 0])
    y_trainl.append(train_data[i:i+prediction_time, 0])




In [235]:
#Convert the x_train and y_train to numpy arrays
x_trainl, y_trainl = np.array(x_trainl), np.array(y_trainl)

In [236]:
#Reshape the data
x_trainl = np.reshape(x_trainl, (x_trainl.shape[0], x_trainl.shape[1], 1))
x_trainl.shape

(21595, 24, 1)

In [237]:
from keras.layers import RepeatVector
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape= (x_trainl.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(y_trainl.shape[1]))

print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 24, 50)            10400     
_________________________________________________________________
lstm_25 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_24 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 26        
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________
None


In [238]:
#Compile the model
model.compile(optimizer='adam',loss='mean_squared_error')

In [239]:
#Train the model
lstm_start = time.time()

history = model.fit(x_trainl, y_trainl, epochs=15, batch_size=32)
lstm_end = time.time()

total_lstm = lstm_end - lstm_start

Epoch 1/15
675/675 [==============================] - 14s 16ms/step - loss: 0.1049
Epoch 2/15
675/675 [==============================] - 11s 16ms/step - loss: 0.0483
Epoch 3/15
675/675 [==============================] - 11s 16ms/step - loss: 0.0460 0s
Epoch 4/15
675/675 [==============================] - 14s 21ms/step - loss: 0.0451
Epoch 5/15
675/675 [==============================] - 17s 25ms/step - loss: 0.0450 0s - los
Epoch 6/15
675/675 [==============================] - 17s 25ms/step - loss: 0.0435
Epoch 7/15
675/675 [==============================] - 15s 23ms/step - loss: 0.0435
Epoch 8/15
675/675 [==============================] - 13s 19ms/step - loss: 0.0435
Epoch 9/15
675/675 [==============================] - 14s 21ms/step - loss: 0.0431
Epoch 10/15
675/675 [==============================] - 13s 19ms/step - loss: 0.0424
Epoch 11/15
675/675 [==============================] - 12s 18ms/step - loss: 0.0421
Epoch 12/15
675/675 [==============================] - 13s 19ms/step - lo

In [240]:
model.save("uni_model_1week.h5")
import pickle
scalerfile = 'scalerweek.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [ ]:
from tensorflow.keras.models import load_model
import pickle
model_uni_day = load_model('multi_model_1hour.h5')
scalerfile = 'scalerhour.sav'
scalerimported = pickle.load(open(scalerfile, 'rb'))

predictions_uni = model_uni_day.predict(X_test[:,:,0:1])

predictions_uni = scalerimported.inverse_transform(predictions_uni)

In [245]:
#Create the testing data set
#Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - training_window: , :]
#Create the data sets x_test and y_test
x_testl = []
y_testl = dataset[training_data_len:, :]
for i in range(training_window, len(test_data)):
    x_testl.append(test_data[i-training_window:i, :])

#Convert the data to a numpy array
x_testl = np.array(x_testl)
print(x_testl.shape)
#Reshape the data
x_testl = np.reshape(x_testl, (x_testl.shape[0], x_testl.shape[1], 1 ))
print(x_testl.shape)
#Get the models predicted price values 
#predictionsl = model_uni_day .predict(x_testl)
# print(print(x_testl[0:1].shape))
# data_csv = pd.read_csv("historical_data.csv")
# data_list=list(data_csv)
# data_list= data_list[-1:]+ data_list[6:-1]
# data_for_prediction=data_csv[data_list[0:1]][-24*7*3:]
# data_for_prediction = np.expand_dims(data_for_prediction, axis=0)
predictionsl = model.predict(x_testl)
predictionsl = scaler.inverse_transform(predictionsl)
print(predictionsl.shape)
predictionsl

(5404, 24, 1)
(5404, 24, 1)
(5404, 1)


array([[44.967678],
       [40.439262],
       [34.168316],
       ...,
       [84.28418 ],
       [82.13445 ],
       [81.72872 ]], dtype=float32)

In [243]:
#predictionsl=scaler.inverse_transform(x_testl[0,:])
#y_testl = dataset[-24*7:, :]

In [249]:
#Get the root mean squared error (RMSE)
rmse=np.sqrt(np.mean(((predictionsl - y_testl)**2)))
rmse

5.650453696891265